# TP : 2-approximation du voyageur de commerce

> Ce TP est à rendre au plus tard ce **dimanche 27 novembre**. Le plus simple est de répondre à ce TP sur [Capytale](https://cas.ent.auvergnerhonealpes.fr/login?service=https%3A//capytale2.ac-paris.fr/web/c-auth/pvd/mcer/connect). J'aurai alors directement accès à votre fichier que je commenterai.  
> [**Lien pour répondre au TP sur Capytale (se connecter avec les identifiants ENT puis cliquer sur Go !).**](https://capytale2.ac-paris.fr/web/c/6116-1007301)  
> Si vous ne pouvez pas vous connecter sur Capytale, vous pouvez terminer le TP sur Basthon, puis télécharger et m'envoyer le notebook, à qpfortier@gmail.com.

<center><img src=https://imgs.xkcd.com/comics/travelling_salesman_problem.png width=60%></center>

---

Soit $G$ un graphe non orienté, complet et pondéré. Le **problème du voyageur de commerce** consiste à trouver un cycle de poids minimum passant par tous les sommets de $G$. Ce problème est NP-difficile, ce qui signifie qu'il n'existe probablement pas d'algorithme efficace (polynomial en la taille de l'entrée) pour le résoudre de façon exacte.  
À la place, on va trouver une solution approchée à ce problème. En effet, nous allons trouver un cycle dont le poids est au plus le double de l'optimal, avec l'algorithme suivant :  
1. Calcul d'un arbre couvrant de poids minimal $T$ dans $G$, avec l'algorithme de Kruskal.  
2. Construction un cycle $C$ en parcourant $T$ en profondeur, en commençant par un sommet quelconque.
  
On admettra que, si $G$ est euclidien (c'est-à-dire que les poids des arêtes vérifient l'inégalité triangulaire - ce qui est le cas pour des coordonnées), alors cet algorithme trouve un cycle de poids au plus le double de l'optimal ([voir une démonstration](https://perso.eleves.ens-rennes.fr/people/julie.parreaux/fichiers_agreg/info_dev/ApproximationTSP.pdf)).

## Graphe des villes

<center><img src=https://raw.githubusercontent.com/mp-info/mp-info.github.io/main/files/tp/tp_tsp/cities.png width=60%></center>

Nous allons tester nos fonctions sur le graphe des 10 plus grandes villes françaises, où le poids entre deux villes est sa distance à vol d'oiseau. Sur la carte ci-dessus, on a affiché le cycle de poids minimum passant par toutes ces villes (on admet qu'il est bien minimum).

Le tableau `cities` contient les coordonnées GPS des villes :

In [2]:
let cities = [|(50.6333, 3.06667); (44.8333, -0.566667); (43.6, 3.88333); (48.5833, 7.75); (47.2167, -1.55); (43.7, 7.25); (43.6, 1.43333); (45.7589, 4.84139); (43.2967, 5.37639); (48.86, 2.34445)|]

val cities : (float * float) array =
  [|(50.6333, 3.06667); (44.8333, -0.566667); (43.6, 3.88333);
    (48.5833, 7.75); (47.2167, -1.55); (43.7, 7.25); (43.6, 1.43333);
    (45.7589, 4.84139); (43.2967, 5.37639); (48.86, 2.34445)|]


Dans la suite, on identifiera les villes par leur indice dans le tableau `cities`.

Pour calculer la distance (en kms) à vol d'oiseau entre deux villes de coordonnées $(\phi_1, \lambda_1)$ et $(\phi_2, \lambda_2)$ (en radians), on utilisera la formule de suivante (distance sur une sphère) :  

$$
R \arccos \left( \sin \phi_1 \sin \phi_2 + \cos \phi_1 \cos \phi_2 \cos (\lambda_2 - \lambda_1) \right)
$$

où $R \approx 6400$ km est le rayon de la Terre.

````{admonition} Question
 Écrire une fonction `deg_to_rad : float -> float` qui convertit un angle en degrés en radians. On utilisera $\pi \approx 3.14$.
````

````{admonition} Solution
:class: tip, dropdown
``` ocaml
let deg_to_rad d = d *. 3.14 /. 180.0
```
``` 
val deg_to_rad : float -> float = <fun>


```
````

````{admonition} Question
 Écrire une fonction `d i j` qui renvoie la distance entre les villes `i` et `j` (en km). On utilisera `cos`, `sin` et `acos` (pour $\arccos$).
````

````{admonition} Solution
:class: tip, dropdown
``` ocaml
let d i j =
  let (phi1, lambda1) = cities.(i) in
  let (phi2, lambda2) = cities.(j) in
  let phi1 = deg_to_rad phi1 in
  let phi2 = deg_to_rad phi2 in
  let lambda1 = deg_to_rad lambda1 in
  let lambda2 = deg_to_rad lambda2 in
  6400. *. acos (sin phi1 *. sin phi2 +. cos phi1 *. cos phi2 *. cos (lambda2 -. lambda1))
```
``` 
val d : int -> int -> float = <fun>


```
````

In [5]:
d 0 8 (* distance entre Lille et Marseille *)

- : float = 837.684895087478253


````{admonition} Question
 Définir une variable `g_cities` qui contient la matrice pondérée du graphe complet dont les sommets sont les villes de `cities` et dont les arêtes ont pour poids la distance à vol d'oiseau entre les villes.
````

````{admonition} Solution
:class: tip, dropdown
``` ocaml
let g_cities = Array.make_matrix 10 10 0.

let () =
  for i = 0 to 9 do
    for j = 0 to 9 do
      g_cities.(i).(j) <- d i j
    done
  done
```
``` 
val g_cities : float array array =
  [|[|0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.|];
    [|0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.|];
    [|0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.|];
    [|0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.|];
    [|0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.|];
    [|0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.|];
    [|0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.|];
    [|0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.|];
    [|0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.|];
    [|0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.|]|]


```
````

On représente un cycle par une liste de sommets. Par exemple, `[0; 3; 7; 5; 8; 2; 6; 1; 4; 9]` est le cycle qui commence à la ville `0` (Lille) puis va à la ville `3` (Strasbourg)... jusqu'à la ville `9` (Paris) et revient à la ville `0` (Lille).

````{admonition} Exercice
 Écrire une fonction `length_cycle : int list -> float` qui renvoie la longueur d'un cycle (en utilisant `g_cities`).  
````

````{admonition} Solution
:class: tip, dropdown
``` ocaml
let length_cycle cycle =
  let debut = List.hd cycle in
  let rec aux = function
    | [] -> 0.
    | [x] -> g_cities.(x).(debut)
    | x :: y :: xs -> g_cities.(x).(y) +. aux (y :: xs) in
  aux cycle
```
``` 
val length_cycle : int list -> float = <fun>


```
````

In [8]:
length_cycle [0; 3; 7; 5; 8; 2; 6; 1; 4; 9]

- : float = 2611.39276745210555


## Tas

Pour implémenter l'algorithme de Kruskal, il faut considérer les arêtes par poids croissant. Pour cela, on va utiliser un tas min.  
Remarque : on pourrait aussi utiliser n'importe quel algorithme de tri, comme dans le cours.

On rappelle qu'un tas min est un arbre presque complet dans lequel chaque noeud est plus petit que ses deux fils. On stocke les sommets du tas min dans un tableau $a$ tel que le fils gauche de `a.(i)` est `a.(2*i+1)` et le fils droit est `a.(2*i+2)`. Le père de `a.(i)` est `a.(i - 1)/2`.  

Voici un exemple de tas min, représenté par le tableau `[|0; 1; 4; 3; 2; 5|]` :
<center><img src=https://raw.githubusercontent.com/fortierq/tikz/master/tree/heap/heap_min/heap_min.png width=30%></center>

Nous allons utiliser le même type que dans le cours :

In [9]:
type 'a heap = {a : 'a array; mutable n : int}

type 'a heap = { a : 'a array; mutable n : int; }


On rappelle que `n` est le nombre d'éléments dans le tas, et que la taille de `a` peut être supérieure à `n` (pour éviter de réallouer à chaque fois qu'on ajoute un élément).

````{admonition} Question
 Écrire une fonction `swap : 'a heap -> int -> int -> unit` qui échange les éléments d'indice `i` et `j` dans le tableau `a` du tas `h`.
````

````{admonition} Solution
:class: tip, dropdown
``` ocaml
let swap h i j =
  let tmp = h.a.(i) in
  h.a.(i) <- h.a.(j);
  h.a.(j) <- tmp
```
``` 
val swap : 'a heap -> int -> int -> unit = <fun>


```
````

````{admonition} Question
 Écrire une fonction `add : 'a heap -> 'a -> unit` qui ajoute un élément à un tas. Il faut :  
- Ajouter l'élément à la fin du tableau.  
- Remonter l'élément jusqu'à ce qu'il soit plus petit que son père (ou qu'il soit à la racine). On pourra utiliser une fonction récursive auxiliaire (équivalente à la fonction `up` du cours).
````

````{admonition} Solution
:class: tip, dropdown
``` ocaml
let add h x =
  let rec aux i =
    let p = (i - 1) / 2 in
    if p >= 0 && h.a.(p) > x then (
      swap h i p;
      aux p
    ) in
  h.a.(h.n) <- x;
  aux h.n;
  h.n <- h.n + 1
```
``` 
val add : 'a heap -> 'a -> unit = <fun>


```
````

````{admonition} Question
 Écrire une fonction `extract_min : 'a heap -> 'a` qui supprime et renvoie le minimum du tas. Il faut :  
- Remplacer la racine par le dernier élément du tableau.  
- Descendre l'élément jusqu'à ce qu'il soit plus petit que ses deux fils (ou qu'il soit une feuille). On pourra utiliser une fonction récursive auxiliaire (équivalente à la fonction `down` du cours).
````

````{admonition} Solution
:class: tip, dropdown
``` ocaml
let extract_min h =
  let rec down i =
    let m = ref i in (* minimum parmi i et ses deux fils *)
    if 2*i + 1 < h.n && h.a.(2*i + 1) < h.a.(!m) then m := 2*i + 1;
    if 2*i + 2 < h.n && h.a.(2*i + 2) < h.a.(!m) then m := 2*i + 2;
    if !m <> i then (
      swap h i !m;
      down !m
    ) in
  let x = h.a.(0) in
  h.n <- h.n - 1;
  h.a.(0) <- h.a.(h.n);
  down 0;
  x
```
``` 
val extract_min : 'a heap -> 'a = <fun>


```
````

````{admonition} Question
 En utilisant les fonctions précédentes, définir un tas min en ajoutant 3, 1, 2, 0 puis en supprimant deux fois le minimum puis en ajoutant 5, 0, 1, 4. Vérifier que vous obtenez le tas représenté sur l'exemple en début de section.
````

In [28]:
let h = {a = Array.make 10 0; n = 0};;
add h 3; add h 1; add h 2; add h 0; h;;
for i = 0 to 1 do ignore (extract_min h) done;
add h 5; add h 0; add h 1; add h 4;
h

val h : int heap = {a = [|0; 0; 0; 0; 0; 0; 0; 0; 0; 0|]; n = 0}


- : int heap = {a = [|0; 1; 2; 3; 0; 0; 0; 0; 0; 0|]; n = 4}


- : int heap = {a = [|0; 1; 4; 3; 2; 5; 0; 0; 0; 0|]; n = 6}


## Algorithme de Kruskal

On rappelle l'algorithme de Kruskal (dans la version utilisant une file de priorité) :  
1 On crée un tas min contenant toutes les arêtes du graphe $G$, sous forme de triplets (poids, sommet de départ, sommet d'arrivée).  
2 On crée un arbre couvrant de poids minimum $T$, en ajoutant les arêtes du tas min dans l'ordre croissant de leur poids, si cela ne crée pas de cycle.

Contrairement à $G$ qui est représenté par une matrice pondérée (comme `g_cities`), $T$ sera représenté par une liste d'adjacence. Ceci est justifié par le fait que $G$ contient beaucoup d'arêtes ($\binom{n}{2}$) alors que $T$ en contient peu (environ $n-1$). De plus, on ne stocke pas les poids dans $T$ (on peut les retrouver dans $G$).

Pour détecter si une arête $\{u, v\}$ crée un cycle, il suffit de regarder (dans $T$) s'il y a un chemin de $u$ à $v$, ce qui peut être fait grâce à un parcours de graphe (DFS ou BFS).

````{admonition} Question
 Écrire une fonction `has_path : int list array -> int -> int -> bool` telle que `has_path t u v` renvoie `true` si et seulement s'il existe un chemin entre les sommets `u` et `v` dans le graphe `t`.
````

````{admonition} Solution
:class: tip, dropdown
``` ocaml
let has_path t u v =
  let n = Array.length t in
  let visited = Array.make n false in
  let rec aux u =
    if not visited.(u) then (
       visited.(u) <- true;
       List.iter aux t.(u)
    ) in
   aux u;
   visited.(v)
```
``` 
val has_path : int list array -> int -> int -> bool = <fun>


```
````

````{admonition} Question
 Écrire une fonction `kruskal : float array array -> int list array` telle que `kruskal g` renvoie un arbre couvrant de poids minimum du graphe `g`.
````

````{admonition} Solution
:class: tip, dropdown
``` ocaml
let kruskal g =
  let n = Array.length g in
  let p = n*n in (* nombre d'arêtes *)
  let h = {a = Array.make p (0., 0, 0); n = 0} in
  for i = 0 to n - 1 do
    for j = 0 to n - 1 do
      add h (g.(i).(j), i, j)
    done
  done;
  let mst = Array.make n [] in
  for i = 0 to p - 1 do
      let (w, u, v) = extract_min h in
      if not (has_path mst u v) then (
        mst.(u) <- v :: mst.(u);
        mst.(v) <- u :: mst.(v);
      )
  done;
  mst
```
``` 
val kruskal : float array array -> int list array = <fun>


```
````

In [14]:
kruskal g_cities

- : int list array =
[|[7; 2; 6; 5; 8]; [3]; [0]; [6; 1]; [5]; [9; 4; 0]; [3; 0]; [0]; [0]; [5]|]


## 2-approximation du voyageur de commerce



````{admonition} Question
 Écrire une fonction `approx_tsp : float array array -> int list` telle que `approx_tsp g` :  
1. Calcule un arbre couvrant de poids minimal `t` dans `g`.  
2. Renvoie la liste des sommets de `t` dans un parcours en profondeur (dans l'ordre préfixe), en commençant par un sommet quelconque.
````

````{admonition} Solution
:class: tip, dropdown
``` ocaml
let approx_tsp g =
  let n = Array.length g in
  let mst = kruskal g in
  let visited = Array.make n false in
  let tour = ref [] in
  let rec aux u =
    if not visited.(u) then (
      visited.(u) <- true;
      tour := u :: !tour;
      List.iter aux mst.(u)
    ) in
  aux 0;
  !tour
```
``` 
val approx_tsp : float array array -> int list = <fun>


```
````

In [17]:
approx_tsp g_cities

- : int list = [8; 4; 9; 5; 1; 3; 6; 2; 7; 0]


````{admonition} Question
 Vérifier que le cycle renvoyé est bien de poids au plus deux fois le poids du cycle optimal.
````

## Pour ceux qui ont fini

Cette partie n'est pas à rendre (sauf si vous avez le temps de le faire...).

````{admonition} Question
 Écrire une fonction `prim : float array array -> int list array` telle que `prim g` renvoie un arbre couvrant de poids minimum du graphe `g`, en utilisant l'algorithme de Prim au lieu de celui de Kruskal (voir la fin du cours pour la description de l'algorithme). Vérifier que le poids de l'arbre couvrant obtenu est le même que celui avec l'algorithme de Kruskal.
````

````{admonition} Question
 Résoudre [ce problème](https://projecteuler.net/problem=83) sur projecteuler, en utilisant l'algorithme de Dijkstra.
````